# What Strategy?
### Based on yelp challenge <a href='https://www.yelp.com.au/dataset/challenge'>dataset </a>

#### Author: Ignacio Recasens

### Abstract

To determine the business strategy that would improve chances of success the most, we performed three analyses that complemented each other. First performing sentiment analysis over customer reviews, second **finding the most significant business attributes that positively correlate to a higher business rating or stars**, and finally performing a qualitative analysis through a conjoint analysis over a survey sent to people residing mainly in Las Vegas.

The purpose of this notebook is to show the steps taken to determine the key **differentiators** for our Ramen Restaurant Chain based using **Regression Analysis** over Yelp customer reviews. In particular we want to find the attributes that have the higher impact on the regression being also statistically significant when predicting business rating. 

In this notebook we have the data transformations needed to have a tidy dataset to apply a regression afterward. This regression is then applied using R in a different notebook (please refer to Part B).


#### Outline

#### 1. <a href='#mysql'>Load Data</a>

#### 2. <a href='#data'>Data Transformations</a>

#### 3. <a href='#viz'>Data Visualization</a>

#### 4. <a href='#reg'>Main attributes by Regresion</a>



### Load libraries

In [1]:
# MySQL
import MySQLdb # For Windows: 'conda.exe install mysql-python' or 'pip install mysqlclient'
import json

# DATA MUNGING
import numpy as np
import math
import pandas as pd
import timeit


# DATA VIZUALIZATION
import seaborn as sns # For Data VIzualization
import matplotlib.pyplot as plt
%matplotlib inline



<a id='mysql'></a>
## 1 MySQL Queries

Let's load the Data from he MySQL Database. 

In [2]:
## QUERY TO EXTRACT CATEGORIES TAblE AND MANUALLY MARK THOSE TO BE SELECTED.

connection = MySQLdb.connect("localhost",'root', 'irecasens_2017', 'yelp_db')
cursor = connection.cursor()

sql = "select category, count(*) from category group by 1;"

cursor.execute(sql)
cat = cursor.fetchall()

cat = pd.DataFrame(list(cat))
cat.columns = ['category', 'N']

cat.to_csv('cat.csv') # For R

connection.close() 


In [3]:
connection = MySQLdb.connect("localhost",'root', 'irecasens_2017', 'yelp_db')
cursor = connection.cursor()

sql = "select business_id, name, value from tbl_V_atributes_NV" # Extract business_id and attributes for restaurants in Las Vegas.
cursor.execute(sql)
att_df = cursor.fetchall()
att_df = pd.DataFrame(list(att_df))
att_df.columns = ['business_id', 'attribute', 'value'] 
display(len(att_df))
display(att_df[5:10])

sql = "select id, city, state, latitude, longitude, stars, review_count, is_open  from tbl_V_business_category_NV"
cursor.execute(sql)
business_df = cursor.fetchall()
business_df = pd.DataFrame(list(business_df))
business_df.columns = ['business_id', 'city', 'state', 'latitude', 'longitude', 'business_stars', 'review_count', 'is_open']
display(len(business_df))
display(business_df[5:10])

connection.close() 


113819

,business_id,attribute,value
5,--9e1ONYQuAa-CB_Rrw7Tw,Ambience,"{""romantic"": false, ""intimate"": false, ""classy..."
6,--9e1ONYQuAa-CB_Rrw7Tw,RestaurantsGoodForGroups,1
7,--9e1ONYQuAa-CB_Rrw7Tw,BYOBCorkage,yes_corkage
8,--9e1ONYQuAa-CB_Rrw7Tw,Caters,0
9,--9e1ONYQuAa-CB_Rrw7Tw,WiFi,no


7303

,business_id,city,state,latitude,longitude,business_stars,review_count,is_open
5,-3H_6UZGWrfl8XQsm1fBmw,Las Vegas,NV,36.1193,-115.146,2.0,5,1
6,-3zffZUHoY8bQjGfPSoBKQ,Las Vegas,NV,36.1120,-115.177,4.0,574,1
7,-46pFijv3f2jZH-_Ze6CQA,Las Vegas,NV,36.1154,-115.178,4.0,20,1
8,-8R_-EkGpUhBk55K9Dd4mg,Las Vegas,NV,36.0725,-115.207,3.5,90,1
9,-8ZiMXZReeTD3kwEvS0Lww,Las Vegas,NV,36.2778,-115.287,4.5,106,1


In [4]:
# EXAMPLE
att_df.loc[(att_df["business_id"] == "--9e1ONYQuAa-CB_Rrw7Tw") & (att_df["attribute"] == "Ambience")]["value"].item()


'{"romantic": false, "intimate": false, "classy": true, "hipster": false, "divey": false, "touristy": false, "trendy": false, "upscale": true, "casual": false}'

Apparently there are some value that are stored as a dicitonary. We need to transform these into a workable dataset. 

<a id='data'></a>
## 2 Data Transformations

In [7]:
temp_df = att_df.copy()
temp_df

,business_id,attribute,value
0,--9e1ONYQuAa-CB_Rrw7Tw,Alcohol,full_bar
1,--9e1ONYQuAa-CB_Rrw7Tw,HasTV,0
2,--9e1ONYQuAa-CB_Rrw7Tw,NoiseLevel,average
3,--9e1ONYQuAa-CB_Rrw7Tw,RestaurantsAttire,dressy
4,--9e1ONYQuAa-CB_Rrw7Tw,BusinessAcceptsCreditCards,1
5,--9e1ONYQuAa-CB_Rrw7Tw,Ambience,"{""romantic"": false, ""intimate"": false, ""classy..."
6,--9e1ONYQuAa-CB_Rrw7Tw,RestaurantsGoodForGroups,1
7,--9e1ONYQuAa-CB_Rrw7Tw,BYOBCorkage,yes_corkage
8,--9e1ONYQuAa-CB_Rrw7Tw,Caters,0
9,--9e1ONYQuAa-CB_Rrw7Tw,WiFi,no


In [8]:
def create_columns(df):
    new_df = df.copy()
    cols = []
    
    for i in range(len(df)):
        val = df.iloc[i,[2]].item()  
        
        start = val.find('{')
        end = val.find('}')
        
        if start >= 0:
            val = val[start:end+1]   
            val = json.loads(val)            
                        
            for k in val.keys():
                new_col = str(df.iloc[i,[1]].item()) + "_" + str(k)
                if new_col not in cols:
                    cols.append(new_col)
                    
        else:
            new_col = str(df.iloc[i,[1]].item())
            if new_col not in cols:
                    cols.append(new_col)
                    
    for col in cols:
        new_df[col] = ""  
                        
    return new_df


def extract_values(df):
    new_df = create_columns(df)
    
    for i in range(len(df)):
        val = df.iloc[i,[2]].item()  
        
        start = val.find('{')
        end = val.find('}')
        
        if start >= 0:
            val = val.replace("false", "0")
            val = val.replace("true", "1")
            val = val[start:end+1]   
            val = json.loads(val)            
                        
            for k, v in val.items():
                col = str(df.iloc[i,[1]].item()) + "_" + str(k)
                new_df[col][i] = str(v)
                
        else:
            col = str(df.iloc[i,[1]].item())
            new_df[col][i] = val
            
    cols = list(new_df)
    to_remove = ["business_id", "attribute", "value"]
    cols = [x for x in cols if x not in to_remove]

    new_df = new_df.groupby(["business_id"],as_index = False)[cols].agg('max')
        
    return new_df


tic = timeit.default_timer()
new_df = extract_values(temp_df)
toc = timeit.default_timer()
print("minutes: ", (toc - tic)/60)

new_df



minutes:  7.089670736833463


,business_id,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,BusinessAcceptsCreditCards,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,...,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms
0,--9e1ONYQuAa-CB_Rrw7Tw,full_bar,0,average,dressy,1,0,0,1,0,...,,,,,,,,,,
1,--q7kSBRb0vWC8lSkXFByA,full_bar,1,average,casual,1,0,0,0,0,...,,,,,,,,,,
2,-0BxAGlIk5DJAGVkpqBXxg,,,,,1,,,,,...,,,,,,,,,,
3,-153AjTW5luZPK4omEujWA,,,,casual,1,,,,,...,,,,,,,,,,
4,-1vfRrlnNnNJ5boOVghMPA,beer_and_wine,1,average,casual,1,0,0,0,0,...,,,,,,,,,,
5,-3H_6UZGWrfl8XQsm1fBmw,,,,,1,,,,,...,,,,,,,,,,
6,-3zffZUHoY8bQjGfPSoBKQ,full_bar,0,average,dressy,1,0,0,1,0,...,,,,,,,,,,
7,-46pFijv3f2jZH-_Ze6CQA,,,,,1,,,,,...,,,,,,,,,,
8,-8R_-EkGpUhBk55K9Dd4mg,none,1,average,casual,1,0,0,0,0,...,,,,,,,,,,
9,-8ZiMXZReeTD3kwEvS0Lww,,,,,1,,,,,...,,,,,,,,,,


In [9]:
# EXAMPLE
display(att_df.loc[(att_df["business_id"] == "--9e1ONYQuAa-CB_Rrw7Tw") & (att_df["attribute"] == "Ambience")]["value"].item())

display(new_df.loc[(new_df["business_id"] == "--9e1ONYQuAa-CB_Rrw7Tw")])


'{"romantic": false, "intimate": false, "classy": true, "hipster": false, "divey": false, "touristy": false, "trendy": false, "upscale": true, "casual": false}'

,business_id,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,BusinessAcceptsCreditCards,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,...,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms
0,--9e1ONYQuAa-CB_Rrw7Tw,full_bar,0,average,dressy,1,0,0,1,0,...,,,,,,,,,,


Now we only need binary variables since we want to do this in a regression.

In [10]:
new_df2 = new_df.copy()
new_df2 = new_df2.replace(np.nan, -1)
new_df2 = new_df2.replace('', -1)
new_df2 = new_df2.replace('no', 0)
new_df2 = new_df2.replace('none', 0)

business_df = new_df2.merge(business_df, left_on='business_id', right_on='business_id', how='left')

display(new_df.head())
business_df.head()

,business_id,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,BusinessAcceptsCreditCards,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,...,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms
0,--9e1ONYQuAa-CB_Rrw7Tw,full_bar,0,average,dressy,1,0,0,1,0,...,,,,,,,,,,
1,--q7kSBRb0vWC8lSkXFByA,full_bar,1,average,casual,1,0,0,0,0,...,,,,,,,,,,
2,-0BxAGlIk5DJAGVkpqBXxg,,,,,1,,,,,...,,,,,,,,,,
3,-153AjTW5luZPK4omEujWA,,,,casual,1,,,,,...,,,,,,,,,,
4,-1vfRrlnNnNJ5boOVghMPA,beer_and_wine,1,average,casual,1,0,0,0,0,...,,,,,,,,,,


,business_id,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,BusinessAcceptsCreditCards,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,...,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,city,state,latitude,longitude,business_stars,review_count,is_open
0,--9e1ONYQuAa-CB_Rrw7Tw,full_bar,0,average,dressy,1,0,0,1,0,...,-1,-1,-1,Las Vegas,NV,36.1232,-115.169,4.0,1389,1
1,--q7kSBRb0vWC8lSkXFByA,full_bar,1,average,casual,1,0,0,0,0,...,-1,-1,-1,Las Vegas,NV,36.0167,-115.173,4.0,7,0
2,-0BxAGlIk5DJAGVkpqBXxg,-1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,Las Vegas,NV,36.1221,-115.168,3.0,38,1
3,-153AjTW5luZPK4omEujWA,-1,-1,-1,casual,1,-1,-1,-1,-1,...,-1,-1,-1,Las Vegas,NV,36.1030,-115.174,3.0,5,0
4,-1vfRrlnNnNJ5boOVghMPA,beer_and_wine,1,average,casual,1,0,0,0,0,...,-1,-1,-1,Las Vegas,NV,36.2813,-115.287,3.0,76,0


In [11]:
business_df[2:10]

,business_id,Alcohol,HasTV,NoiseLevel,RestaurantsAttire,BusinessAcceptsCreditCards,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,...,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,city,state,latitude,longitude,business_stars,review_count,is_open
2,-0BxAGlIk5DJAGVkpqBXxg,-1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,Las Vegas,NV,36.1221,-115.168,3.0,38,1
3,-153AjTW5luZPK4omEujWA,-1,-1,-1,casual,1,-1,-1,-1,-1,...,-1,-1,-1,Las Vegas,NV,36.1030,-115.174,3.0,5,0
4,-1vfRrlnNnNJ5boOVghMPA,beer_and_wine,1,average,casual,1,0,0,0,0,...,-1,-1,-1,Las Vegas,NV,36.2813,-115.287,3.0,76,0
5,-3H_6UZGWrfl8XQsm1fBmw,-1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,Las Vegas,NV,36.1193,-115.146,2.0,5,1
6,-3zffZUHoY8bQjGfPSoBKQ,full_bar,0,average,dressy,1,0,0,1,0,...,-1,-1,-1,Las Vegas,NV,36.1120,-115.177,4.0,574,1
7,-46pFijv3f2jZH-_Ze6CQA,-1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,Las Vegas,NV,36.1154,-115.178,4.0,20,1
8,-8R_-EkGpUhBk55K9Dd4mg,0,1,average,casual,1,0,0,0,0,...,-1,-1,-1,Las Vegas,NV,36.0725,-115.207,3.5,90,1
9,-8ZiMXZReeTD3kwEvS0Lww,-1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,Las Vegas,NV,36.2778,-115.287,4.5,106,1


<a id='viz'></a>
## 3 Data Visualization

In [ ]:

def the_plot(df, var):    

    display(pd.DataFrame(df.groupby([var])['business_id'].count()).reset_index())
    
    labels = list(pd.DataFrame(df.groupby([var])['business_id'].count()).reset_index()[var])
    i = 0
    for label in labels: 
        if str(label) == "0":
            labels[i] = "No_" + var 
        elif str(label) == "-1":
            labels[i] = "No Information" 
        elif str(label) == "1":
            labels[i] = "With_" + var 
            
        i+=1

    sns.countplot(y = var, data = business_df)
    
    plt.show()
    
    return None
  

def create_plots(df, colnames):
    for var in colnames: 
        the_plot(df, var)
    
    return None
    

to_rmv = ['BYOB', 'BYOBCorkage', 'city',  'latitude', 'longitude', 'business_stars', 'review_count',
         "CoatCheck",  "DriveThru", "GoodForDancing", "HappyHour", 'BusinessAcceptsCreditCards',
         "Open24Hours", "RestaurantsAttire", "RestaurantsCounterService", "RestaurantsTakeOut", "Smoking",
         "state", 'Ambience_romantic', 'Corkage', 'Music_karaoke','Music_jukebox','Music_video', 'Music_live','BusinessParking_validated', 'Music_dj', 'Music_no_music'] # not enough data or irrelevant

plot_business_df = business_df.copy()
plot_business_df.drop(to_rmv, axis = 1, inplace = True)

colnames = list(plot_business_df)
colnames.remove('business_id')

create_plots(plot_business_df, colnames)


<a id='reg'></a>
## 4 Main attributes by Regresion

In [12]:
to_rmv = ["business_id",'BYOB', 'BYOBCorkage', 'city',  'latitude', 'longitude', 'review_count',
         "CoatCheck","DriveThru", "GoodForDancing",  "HappyHour", "Open24Hours", "RestaurantsAttire", "RestaurantsCounterService", "RestaurantsTakeOut", "Smoking",
         "state", "is_open", "BusinessAcceptsCreditCards"]

reg_df = business_df.copy()
reg_df.drop(to_rmv, axis = 1, inplace = True)

reg_df = reg_df.replace(-1, "No")
reg_df = reg_df.replace(0, "No")
reg_df = reg_df.replace(1, "Yes")
reg_df = reg_df.replace("-1", "No")
reg_df = reg_df.replace("0", "No")
reg_df = reg_df.replace("1", "Yes")

reg_df

,Alcohol,HasTV,NoiseLevel,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_hipster,Ambience_divey,Ambience_touristy,Ambience_trendy,...,DietaryRestrictions_vegetarian,HairSpecializesIn_coloring,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_perms,HairSpecializesIn_kids,HairSpecializesIn_extensions,HairSpecializesIn_asian,HairSpecializesIn_straightperms,business_stars
0,full_bar,No,average,No,No,Yes,No,No,No,No,...,No,No,No,No,No,No,No,No,No,4
1,full_bar,Yes,average,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,4
2,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,3
3,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,3
4,beer_and_wine,Yes,average,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,3
5,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,2
6,full_bar,No,average,No,No,Yes,No,No,No,No,...,No,No,No,No,No,No,No,No,No,4
7,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,4
8,No,Yes,average,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,3.5
9,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,4.5


In [13]:
list(reg_df)

['Alcohol',
 'HasTV',
 'NoiseLevel',
 'Ambience_romantic',
 'Ambience_intimate',
 'Ambience_classy',
 'Ambience_hipster',
 'Ambience_divey',
 'Ambience_touristy',
 'Ambience_trendy',
 'Ambience_upscale',
 'Ambience_casual',
 'RestaurantsGoodForGroups',
 'Caters',
 'WiFi',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'Corkage',
 'GoodForKids',
 'WheelchairAccessible',
 'BikeParking',
 'OutdoorSeating',
 'RestaurantsPriceRange2',
 'RestaurantsDelivery',
 'GoodForMeal_dessert',
 'GoodForMeal_latenight',
 'GoodForMeal_lunch',
 'GoodForMeal_dinner',
 'GoodForMeal_breakfast',
 'GoodForMeal_brunch',
 'BusinessParking_garage',
 'BusinessParking_street',
 'BusinessParking_validated',
 'BusinessParking_lot',
 'BusinessParking_valet',
 'Music_dj',
 'Music_background_music',
 'Music_no_music',
 'Music_karaoke',
 'Music_live',
 'Music_video',
 'Music_jukebox',
 'BestNights_monday',
 'BestNights_tuesday',
 'BestNights_friday',
 'BestNights_wednesday',
 'BestNights_thursday',
 'BestNights

In [237]:
colnames = list(reg_df)
colnames.remove("business_stars")

reg_df2 = pd.get_dummies(reg_df, columns =colnames)

reg_df2["business_stars"] = reg_df["business_stars"]
reg_df2 = reg_df2[pd.notnull(reg_df2['business_stars'])]

reg_df2.to_csv('reg_df2.csv') # For R
reg_df2


,business_stars,Alcohol_No,Alcohol_beer_and_wine,Alcohol_full_bar,HasTV_No,HasTV_Yes,NoiseLevel_No,NoiseLevel_average,NoiseLevel_loud,NoiseLevel_quiet,...,HairSpecializesIn_curly_No,HairSpecializesIn_curly_Yes,HairSpecializesIn_perms_No,HairSpecializesIn_kids_No,HairSpecializesIn_kids_Yes,HairSpecializesIn_extensions_No,HairSpecializesIn_extensions_Yes,HairSpecializesIn_asian_No,HairSpecializesIn_asian_Yes,HairSpecializesIn_straightperms_No
0,4,0,0,1,1,0,0,1,0,0,...,1,0,1,1,0,1,0,1,0,1
1,4,0,0,1,0,1,0,1,0,0,...,1,0,1,1,0,1,0,1,0,1
2,3,1,0,0,1,0,1,0,0,0,...,1,0,1,1,0,1,0,1,0,1
3,3,1,0,0,1,0,1,0,0,0,...,1,0,1,1,0,1,0,1,0,1
4,3,0,1,0,0,1,0,1,0,0,...,1,0,1,1,0,1,0,1,0,1
5,2,1,0,0,1,0,1,0,0,0,...,1,0,1,1,0,1,0,1,0,1
6,4,0,0,1,1,0,0,1,0,0,...,1,0,1,1,0,1,0,1,0,1
7,4,1,0,0,1,0,1,0,0,0,...,1,0,1,1,0,1,0,1,0,1
8,3.5,1,0,0,0,1,0,1,0,0,...,1,0,1,1,0,1,0,1,0,1
9,4.5,1,0,0,1,0,1,0,0,0,...,1,0,1,1,0,1,0,1,0,1


In [257]:
display(len(reg_df2.loc[reg_df2["RestaurantsDelivery_No"] == 0]))
display(len(reg_df2.loc[reg_df2["RestaurantsDelivery_No"] == 1]))


1171

6038